In [ ]:
#preprocessing: downsampling, extracting epochs
#plotting

In [ ]:
import database.neuralynx_extract.combinato_tools as ct
import database.neuralynx_extract.nlxio as nlxio

import preprocessing.data_preprocessing.refractor_utils as ru
import sys

import matplotlib.pyplot as plt
import datajoint as dj
import neurodsp.filt
import scipy
from pathlib import Path
import neurokit as nk
import neurokit2 as nk2
import matplotlib.gridspec as gridspec
import time
import os
from pathlib import Path

# Local application imports 
from database.db_setup import *
import preprocessing.data_preprocessing.binning as binning
import preprocessing.data_preprocessing.create_vectors_from_time_points as create

import utils.helper_func as hf
import utils.preprocessing as preprocessing
import utils.plotting as plotting

In [2]:
top_dir = '/home/anastasia/epiphyte/anastasia/output'
ncs_dir = '/home/anastasia/epiphyte/lfp_data'
folder = '01-preprocessed'
downs_dir = f'{top_dir}/{folder}/downsampled'

#46
patient_id = 53
session_nr = 1

df_patient_info = pd.read_csv(f'{top_dir}/{patient_id}_channel_info.csv')

fs = 32768
#The resulting sample rate is up / down times the original sample rate.
up = 1
down=32
fs_downs = (up/down)*fs
dt = 1/fs_downs      # sampling period/time/interval or time resolution, often denoted as T

pos, stim_id, fn, stim_name, mov, paradigm, stim_time = get_screening_data(patient_id, session_nr)
df_stim_info = pd.DataFrame(ScreeningAnnotation())
df_stim_info = df_stim_info.loc[df_stim_info['patient_id'] == patient_id]
df_stim_info = df_stim_info.reset_index()

all_stim = np.unique(df_stim_info['stim_id'])
all_stim_name = np.unique(df_stim_info['stim_name'])

In [3]:
df_stim_info

,index,patient_id,session_nr,stim_index,position,stim_id,filename,stim_name,is_500_days,paradigm,time
0,1680,53,1,0,pre,16,name-tom.jpg,Tom,1,nscr,5.401209e+08
1,1681,53,1,1,pre,34,tom3.jpg,Tom,1,scr,5.401263e+08
2,1682,53,1,2,pre,4,elaine.jpg,The Graduate,0,scr,5.401289e+08
3,1683,53,1,3,pre,17,name-vance.jpg,Vance,1,nscr,5.401310e+08
4,1684,53,1,4,pre,13,name-paul.jpg,Paul,1,nscr,5.401332e+08
...,...,...,...,...,...,...,...,...,...,...,...
835,2515,53,1,835,post,41,woman1.jpg,A Woman,1,scr,5.468749e+08
836,2516,53,1,836,post,18,paul1.jpg,Paul,1,scr,5.468761e+08
837,2517,53,1,837,post,3,casablanca.jpg,Casablanca,0,scr,5.468773e+08
838,2518,53,1,838,post,4,elaine.jpg,The Graduate,0,scr,5.468786e+08


In [3]:
#downsample
#10 mins

ch='CSC1'

t = time.time()
my_file = Path(f'{downs_dir}/{patient_id}_channel-{ch}_downsampled_data.npy')

if not os.path.exists(my_file):
    os.makedirs(downs_dir)

if my_file.is_file():
    print('Downsampling exists. Load downsampled data file')
    timestamps_downs_patient = np.load(f'{downs_dir}/{patient_id}_timestamps_downs.npy')
    
else:
    preprocessing.downsample(ncs_dir, patient_id, downs_dir, df_patient_info['channel_name'], up, down)
    
elapsed = time.time() - t
print(f'Elapsed time for downsampling: {elapsed}')

Create downsampled data file
Elapsed time for downsampling: 536.4129498004913


In [4]:
#extract epochs [-1000, 1000]
#we need 2 seconds of data with sampling rate of 1024 -> we need 2048 data points
epoch_len = 2049
timestamps_downs = np.load(f'{downs_dir}/{patient_id}_timestamps_downs.npy')

filtering_types = ['broadband', 'slow_gamma', 'fast_gamma', 'theta', 'lowpass_20']


for filtering_type in filtering_types:
    print(filtering_type)
    for i in range(len(df_patient_info['channel_name'])):
        ch = df_patient_info.loc[i,'channel_name']
        ch_site = df_patient_info.loc[i,'recording_site']
        
        downs_data_file = Path(f'{downs_dir}/{patient_id}_channel-{ch}_downsampled_data.npy')
        downsampled_data = np.load(downs_data_file) 
        
        save_dir = f'{top_dir}/{folder}/epochs/{filtering_type}_2_sec/'
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        epochs, stim_onsets_downs, ofset_ms = preprocessing.create_epochs_array(downsampled_data, timestamps_downs, df_stim_info, patient_id, ch, save_dir, epoch_len, filtering_type=filtering_type, notch_filtering=True)


broadband
slow_gamma
fast_gamma
theta
lowpass_20


In [6]:
plotting.plot_epochs(df_patient_info, patient_id, df_stim_info, all_stim)

In [5]:
plotting.plot_epochs_organized(df_patient_info, patient_id, df_stim_info, all_stim_name)